In [1]:
import json
import pickle
import zipfile

!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 4.1 MB/s 


In [2]:
import pandas as pd

In [3]:
local_zip = "/content/newDataset.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("dataset_new")
zip_ref.close()

In [4]:
type_file = "train"
path = "/content/FaceM-Train.json"
data_file = open(path)
data = json.load(data_file)
data

[{'id': '1fd96e2c-6503-4be6-a855-46cb8d2816eda7379b42-d7ac-4032-b24e-0fad85b041bd',
  'image': 'kids.jpeg',
  'width': 259,
  'height': 194,
  'segmentation_url': '',
  'classification': [],
  'tags': [{'parent': None,
    'color': '#f92552',
    'pos': {'x': 110.14013617962587,
     'y': 39.06833098982876,
     'w': 23.15431551202903,
     'h': 29.309260141808913},
    'classes': [],
    'name': 'MASK',
    'id': 'c87f297a-a4b2-44fa-af06-a4f6b356a540',
    'text': None,
    'type': 'bounding_box'},
   {'parent': None,
    'color': '#f92552',
    'pos': {'x': 169.93102686891604,
     'y': 46.39564602528099,
     'w': 21.981945106356676,
     'h': 28.723074938972722},
    'classes': [],
    'name': 'MASK',
    'id': 'ee4b3bf1-47cc-4ca5-b7f5-9ee9fcdebb09',
    'text': None,
    'type': 'bounding_box'}]},
 {'id': '1fd96e2c-6503-4be6-a855-46cb8d2816ede836136b-65d1-4c53-ae29-339d5c7c0ba4',
  'image': 'many_ppl.jpeg',
  'width': 320,
  'height': 157,
  'segmentation_url': '',
  'classificati

In [5]:
csv_list = []

for classification in data:
    width, height = classification['width'], classification['height']
    image = classification['image'].replace("+", " ")
    
    for item in classification['tags']:
        name = item['name']
        xmin = item['pos']['x']
        ymin = item['pos']['y']
        xmax = item['pos']['x'] + item['pos']['w']
        ymax = item['pos']['y'] + item['pos']['h']

        value = (image, width, height, name, xmin, ymin, xmax, ymax)
        csv_list.append(value)
    
column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 
               'xmax', 'ymax']
csv_df = pd.DataFrame(csv_list, columns = column_name)

csv_df.to_csv("/content/{}_labels.csv".format(type_file))

In [6]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
/content/models
Note: checking out '58d19c67e1d30d905dd5c6e5092348658fed80af'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 58d19c67e Internal change
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86

In [8]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


# Es fundamental remplazar las clases con las mismas clases del proyecto.
# Los nombres deben estar escritos exactamente igual.
# Si existen más clases en tu proyecto agregarias un ELIF.
def class_text_to_int(row_label):
    if row_label == "MASK": 
      return 1
    elif row_label == "NO_MASK":
      return 2
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Modificamos con la ubicación de nuestros archivos. Recuerda tomarte el tiempo 
# de revisar que la URL sea la misma que la de Google Colab en donde subiste el 
# archivo de forma manual
output_path = "train.record"
image_dir = "/content/dataset_new/newDataset"
csv_input = "/content/train_labels.csv"

writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(image_dir)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')

for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

Successfully created the TFRecords: /content/models/research/train.record


In [ ]:
--!zip -r /content/dataset_facemask_final_GOOD.zip /content/dataset_original

In [9]:
type_file = "test"
path = "/content/FaceM-Test.json"
data_file = open(path)
data = json.load(data_file)
data

[{'id': 'e78801d3-3d46-4358-a069-095d0d90cbbdf5b1c77a-5a52-4754-aa10-c62a0d2e8fb2',
  'image': 'bus.jpeg',
  'width': 275,
  'height': 183,
  'segmentation_url': '',
  'classification': [],
  'tags': [{'parent': None,
    'color': '#f92552',
    'pos': {'x': 138.5, 'y': 51, 'w': 39, 'h': 60},
    'classes': [],
    'name': 'MASK',
    'id': '64fc626e-7986-4bfd-a6ff-a3c0668365df',
    'text': None,
    'type': 'bounding_box'}]},
 {'id': 'e78801d3-3d46-4358-a069-095d0d90cbbd0c8c5469-05ae-4495-ba51-78209d830f20',
  'image': 'classroom.jpeg',
  'width': 275,
  'height': 183,
  'segmentation_url': '',
  'classification': [],
  'tags': [{'parent': None,
    'color': '#f92552',
    'pos': {'x': 118.09782331910633,
     'y': 35.76129011275297,
     'w': 28.0716639477762,
     'h': 53.80402256657106},
    'classes': [],
    'name': 'MASK',
    'id': 'ca832018-fb7d-4627-b5cd-3a4970ed56e0',
    'text': None,
    'type': 'bounding_box'},
   {'parent': None,
    'color': '#f92552',
    'pos': {'x':

In [10]:
csv_list = []

for classification in data:
    width, height = classification['width'], classification['height']
    image = classification['image'].replace("+", " ")
    
    for item in classification['tags']:
        name = item['name']
        xmin = item['pos']['x']
        ymin = item['pos']['y']
        xmax = item['pos']['x'] + item['pos']['w']
        ymax = item['pos']['y'] + item['pos']['h']

        value = (image, width, height, name, xmin, ymin, xmax, ymax)
        csv_list.append(value)
    
column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 
               'xmax', 'ymax']
csv_df = pd.DataFrame(csv_list, columns = column_name)

csv_df.to_csv("/content/{}_labels.csv".format(type_file))

In [11]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


# Es fundamental remplazar las clases con las mismas clases del proyecto.
# Los nombres deben estar escritos exactamente igual.
# Si existen más clases en tu proyecto agregarias un ELIF.
def class_text_to_int(row_label):
    if row_label == "MASK": 
      return 1
    elif row_label == "NO_MASK":
      return 2
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Modificamos con la ubicación de nuestros archivos. Recuerda tomarte el tiempo 
# de revisar que la URL sea la misma que la de Google Colab en donde subiste el 
# archivo de forma manual
output_path = "test.record"
image_dir = "/content/dataset_new/newDataset"
csv_input = "/content/test_labels.csv"

writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(image_dir)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')

for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

Successfully created the TFRecords: /content/models/research/test.record
